# 환경 변수 로드

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 3-1

In [4]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 인스턴스를 생성 
model = ChatOpenAI(model="gpt-4o-mini")

# 생성된 모델 인스턴스를 출력
print(model)

client=<openai.resources.chat.completions.Completions object at 0x11a4e2710> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11a4ea6d0> root_client=<openai.OpenAI object at 0x11a4af710> root_async_client=<openai.AsyncOpenAI object at 0x11a4e28d0> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')


# 3-2

In [6]:
from langchain_core.messages import HumanMessage

# HumanMessage 객체를 생성하여 사용자의 입력을 표현
response = model.invoke([HumanMessage(content="안녕하세요! 제 이름은 판다스입니다.")])

# AI의 응답 객체에서 텍스트(content)를 추출 
print("AI:", response.content)

AI: 안녕하세요, 판다스님! 반갑습니다. 어떻게 도와드릴까요?


# 3-3

In [8]:
from langchain_core.prompts import ChatPromptTemplate


# 프롬프트 템플릿 정의 - 시스템 메시지 설정 
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 한국어 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])

# 프롬프트 템플릿과 모델을 연결하여 체인 생성
chain = prompt | model

# 체인 실행
response = chain.invoke({
    "user_input": "한국의 전통 음식 중 하나를 소개해 주세요."
})

# AI의 응답 객체에서 텍스트(content)를 추출 
print("AI:", response.content)

AI: 한국의 전통 음식 중 하나로 **비빔밥**을 소개할 수 있습니다. 비빔밥은 밥 위에 다양한 나물, 고기, 계란, 고추장 등을 올리고 잘 섞어서 먹는 요리입니다. 각 재료의 색깔과 맛이 조화를 이루어 시각적으로도 아름답고, 영양가도 높습니다. 비빔밥은 지역에 따라 다양한 변형이 있으며, 대표적으로 전주비빔밥이 유명합니다. 이 음식은 한국의 대표적인 한 그릇 음식으로, 건강하고 맛있어서 많은 사람들이 사랑합니다.


# 3-4

In [13]:
import gradio as gr

# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수
def answer_invoke(message, history):
    # chain.invoke()를 사용하여 AI 모델에 메시지 전달 및 응답 생성
    response = chain.invoke(message)
    # 생성된 응답 반환
    return response.content

# Gradio ChatInterface 객체 생성
# fn: 메시지 처리 함수
# title: 채팅 인터페이스의 제목
demo = gr.ChatInterface(fn=answer_invoke, title="한국어 Bot")

# Gradio 인터페이스 실행
# launch() 메서드는 웹 서버를 시작하고 채팅 인터페이스를 브라우저에 표시
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [14]:
# Gradio 인터페이스 종료 
demo.close ()

Closing server running on port: 7860


# 3-5

In [15]:
from langchain_core.prompts import ChatPromptTemplate


# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 한국어 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])

# 프롬프트 템플릿과 모델을 연결하여 체인 생성
chain = prompt | model

# 스트리밍 방식으로 체인 실행
for chunk in chain.stream({
    "user_input": "서울의 유명한 관광지 세 곳을 추천해 주세요."
}):
    print(chunk.content, end="")

서울에는 많은 유명한 관광지가 있지만, 그 중에서 세 곳을 추천해드릴게요.

1. **경복궁**: 조선 왕조의 주요 궁궐로, 한국 전통 건축의 아름다움을 감상할 수 있는 곳입니다. 경복궁 내의 국립민속박물관과 국립고궁박물관도 함께 방문할 수 있습니다. 궁궐의 입장료는 저렴하고, 특히 수문장 교대식은 놓치지 말아야 할 볼거리입니다.

2. **남산서울타워 (N 서울타워)**: 서울의 랜드마크 중 하나로, 남산에 위치해 있어 서울 전경을 한눈에 감상할 수 있습니다. 타워에 올라가면 360도 전망을 즐길 수 있으며, 특히 야경이 아름답습니다. 주변에는 남산공원도 있어 산책하기에도 좋은 장소입니다.

3. **명동**: 쇼핑과 맛집으로 유명한 서울의 대표적인 상업 지역입니다. 다양한 브랜드 매장과 길거리 음식, 그리고 카페들이 즐비해 있어 관광객들에게 인기가 많습니다. 특히, 명동 성당과 같은 역사적인 건물도 가까이 있어 방문할 가치가 있습니다.

이 외에도 서울에는 많은 매력적인 장소가 있으니, 방문 계획에 맞춰 다양한 곳을 탐방해보세요!

# 3-6

In [18]:
import gradio as gr

# 스트리밍 방식으로 AI 응답을 생성하는 함수
def answer_stream(message, history):
    partial_message = ""
    # chain.stream()을 사용하여 AI 모델의 응답을 청크 단위로 스트리밍
    for chunk in chain.stream(message):
        partial_message += chunk.content
        # yield를 사용하여 부분적인 응답을 실시간으로 반환
        yield partial_message

# Gradio ChatInterface 객체 생성
# fn: 메시지 처리 함수 (스트리밍 방식)
# title: 채팅 인터페이스의 제목
# examples: 사용자가 시도해볼 수 있는 예제 질문들
demo = gr.ChatInterface(
    fn=answer_stream, 
    title="한국어 Bot",
    examples=[
        "한국의 전통 음식 추천해주세요", 
        "서울의 유명한 관광지는?",
        ],
    retry_btn="다시 생성",
    undo_btn="되돌리기",
    clear_btn="대화 지우기"
)

# Gradio 인터페이스 실행
# launch() 메서드는 웹 서버를 시작하고 채팅 인터페이스를 브라우저에 표시
# share=True로 설정하면 임시 공개 URL을 생성하여 다른 사람과 공유 가능
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://0ca96dc34077209b60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [19]:
# Gradio 인터페이스 종료 
demo.close ()

Closing server running on port: 7860
